In [1]:
import os

In [2]:
imdb_dir = 'data/imdb'

In [3]:
train_dir = os.path.join(imdb_dir,'train')

In [4]:
labels = []

In [5]:
texts = []

In [6]:
for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir,label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            with open(os.path.join(dir_name,fname)) as inputfile:
                texts.append(inputfile.read())
                if label_type == 'neg':
                    labels.append(0)
                else:
                    labels.append(1)

FileNotFoundError: [Errno 2] No such file or directory: 'data/imdb/train/neg'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

In [ ]:
tokenizer = Tokenizer(num_words=maxlen)

In [ ]:
tokenizer.fit_on_texts(texts)

In [ ]:
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
data = pad_sequences(sequences,maxlen=maxlen)

In [ ]:
data.shape

In [ ]:
len(labels)

In [ ]:
import numpy as np

In [ ]:
labels = np.asarray(labels)

In [ ]:
labels.shape

In [ ]:
indices = np.arange(data.shape[0])

In [ ]:
np.random.shuffle(indices)

In [ ]:
data = data[indices]

In [ ]:
labels = labels[indices]

In [ ]:
train_x = data[:training_samples]

In [ ]:
train_y = labels[:training_samples]

In [ ]:
val_x = data[training_samples:training_samples + validation_samples]

In [ ]:
val_y = labels[training_samples:training_samples + validation_samples]

In [ ]:
train_x.shape

In [ ]:
val_x.shape

In [ ]:
val_y.shape

In [ ]:
glove_dir = 'data/glove'
embeddings_index = {}
with open(os.path.join(glove_dir,'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:],dtype='float32')
        embeddings_index[word] = coefs
        
        

In [ ]:
len(embeddings_index)

In [ ]:
embedding_dim = 100

In [ ]:
word_index = tokenizer.word_index

In [ ]:
embedding_matrix = np.zeros((max_words,embedding_dim))

In [ ]:
for word , i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix[1]

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential()

In [ ]:
model.add(tf.keras.layers.Embedding(max_words,embedding_dim , input_length = maxlen))

In [ ]:
model.add(tf.keras.layers.Flatten())

In [ ]:
model.add(tf.keras.layers.Dense(32 , activation='relu'))

In [ ]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
train_y.shape
train_y = train_y.reshape((-1,1))
val_y = val_y.reshape((-1,1))

In [ ]:
model.layers[0].set_weights([embedding_matrix])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',
             metrics=['acc'])

In [ ]:
model.fit(train_x,train_y , epochs = 10 , batch_size=20,validation_data=(val_x,val_y))